In [1]:
#Importing Libraries
import pandas as pd
import numpy as np
from datetime import datetime as dt
from scipy.stats import ttest_ind
import random 
import matplotlib.pyplot as plt 
import plotly.express as px
import seaborn as sns
import xlsxwriter
from pandas import ExcelWriter
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
#GROUP AOV 120K
query = '''

SELECT _id, fromcity, coupon, segment_name, user_id, date(createdAt, 'Asia/Saigon') as dates
, time(createdAt, 'Asia/Saigon') as createdAt, discountedTotalAmount, dishTotalPrice, status
, toLocation_0, toLocation_1, originalTotalPrice,
  case 
    when coupon_code_0 like ('QUAT330120%') then 'QUAT330120'
    when coupon_code_0 like ('QUAT340120%') then 'QUAT340120'
    when coupon_code_0 like ('QUAT350120%') then 'QUAT350120'
    when coupon_code_0 like ('QUAT360120%') then 'QUAT360120'
    when coupon_code_0 like ('QUAT340150%') then 'QUAT340150'
    when coupon_code_0 like ('QUAT350150%') then 'QUAT350150'
    when coupon_code_0 like ('QUAT360150%') then 'QUAT360150'
    when coupon_code_0 like ('QUAT370150%') then 'QUAT370150'
    when coupon_code_0 like ('QUAT350200%') then 'QUAT350200'
    when coupon_code_0 like ('QUAT360200%') then 'QUAT360200'
    when coupon_code_0 like ('QUAT370200%') then 'QUAT370200'
    when coupon_code_0 like ('QUAT380200%') then 'QUAT380200'
    when coupon_code_0 like ('NHOBAN%') then 'NHOBAN'
    when coupon_code_0 like ('BAEMINMM%') then 'BAEMINMM'
    when coupon_code_0 like ('LIXINAMMOI%') then 'LIXINAMMOI'
    when coupon_code_0 like 'MEOMAP%' then 'MEOMAP'
    when (coupon_code_0 like 'TRUMCUOI%' and coupon_code_0 <> 'TRUMCUOIDONGGIA') then 'TRUMCUOI'
    when coupon_code_0 like 'ZALOPAY%' then 'ZALOPAY'
    else coupon_code_0 
  end as coupon_code_0
from dept.personalized_coupon_20210325
join baemin.orders on userId = user_id
where status = 'DELIVERED'
  and createdAt > '2021-02-02'
  and merchantType = 'MERCHANT'
--  and case 
--    when coupon in ('QUAT330120xxxx','QUAT340120xxxx','QUAT350120xxxx','QUAT360120xxxx') then dishTotalPrice >=120000
--    when coupon in ('QUAT340150xxxx','QUAT350150xxxx','QUAT360150xxxx','QUAT370150xxxx') then dishTotalPrice >=150000
--    when coupon in ('QUAT350200xxxx','QUAT360200xxxx','QUAT370200xxxx','QUAT380200xxxx') then dishTotalPrice >=200000
--    else coupon ='CONTROL' 
--    end 
'''
#aov = pd.read_gbq(query,project_id = 'baemin-vietnam',dialect='standard')
aov = pd.read_csv('aov.csv')

In [69]:
citylist=['Hồ Chí Minh','Hà Nội']
groupcp=[120,150,200]
val={120:'120xxxx|CONTROL', 150:'150xxxx|CONTROL', 200:'200xxxx|CONTROL'}
short={'Hồ Chí Minh':'HCM', 'Hà Nội':'HN'}
price={120:120000, 150:150000, 200:200000}
for i in groupcp:
        aovg = aov.loc[aov['coupon'].str.contains(val.get(i)) & aov['dishTotalPrice'].ge(price.get(i))]
        for j in citylist:
            aovcity=aovg[aovg.fromcity==j]   
            aov_cpo=aovcity.groupby('coupon').agg(Cost=('discountedTotalAmount','sum'),order_no=('_id','count')).reset_index()
            aov_cpo['cost_increment']=  (aov_cpo.loc[1:, 'Cost'] -  aov_cpo.at[0, 'Cost'] )/1000
            aov_cpo['order_increment']=  (aov_cpo.loc[1:, 'order_no'] -  aov_cpo.at[0, 'order_no'] )
            aov_cpo['norm'] = MinMaxScaler().fit_transform(np.array(aov_cpo.order_increment).reshape(-1,1)) - MinMaxScaler().fit_transform(np.array(aov_cpo.cost_increment).reshape(-1,1))          
            aov_cpo=aov_cpo.sort_values(by=['norm'], ascending=False).fillna(0)                                                                           
            aov_cpo.rename(columns={"coupon":"{} Coupon".format(i)}, inplace=True)
            aov_cpo.rename(columns={"Cost":"{} Cost".format(short.get(j))}, inplace=True)
            print(aov_cpo)

       120 Coupon   HCM Cost  order_no  cost_increment  order_increment  \
3  QUAT350120xxxx  284973080      6437      104661.306           1750.0   
2  QUAT340120xxxx  215248160      5354       34936.386            667.0   
1  QUAT330120xxxx  189951820      4975        9640.046            288.0   
4  QUAT360120xxxx  355194608      7120      174882.834           2433.0   
0         CONTROL  180311774      4687           0.000              0.0   

           norm  
3  1.065448e-01  
2  2.360408e-02  
1  0.000000e+00  
4 -2.220446e-16  
0  0.000000e+00  
       120 Coupon    HN Cost  order_no  cost_increment  order_increment  \
3  QUAT350120xxxx  152176220      3303        64835.18            986.0   
2  QUAT340120xxxx  110912480      2750        23571.44            433.0   
1  QUAT330120xxxx   97746080      2529        10405.04            212.0   
4  QUAT360120xxxx  196852820      3835       109511.78           1518.0   
0         CONTROL   87341040      2317            0.00            

In [80]:
citylist=['Hồ Chí Minh','Hà Nội']
groupcp=[120,150,200]
val={120:'120xxxx|CONTROL', 150:'150xxxx|CONTROL', 200:'200xxxx|CONTROL'}
short={'Hồ Chí Minh':'HCM', 'Hà Nội':'HN'}
price={120:120000, 150:150000, 200:200000}
for i in groupcp:
        aovg = aov.loc[aov['coupon'].str.contains(val.get(i)) & aov['dishTotalPrice'].ge(price.get(i))]
        for j in citylist:
            aovcity=aovg[aovg.fromcity==j]   
            aov_cpo=aovcity.groupby('coupon').agg(Cost=('discountedTotalAmount','sum'),order_no=('_id','count')).reset_index()
            aov_cpo['CPO']=  aov_cpo.Cost/aov_cpo.order_no
            aov_cpo['order_increment']=  aov_cpo['order_no'].diff().fillna(0)  
            aov_cpo['cpo_increment']=  aov_cpo['CPO'].diff().fillna(0) 
                   
            # Create figure with CPO and or increment plot
            fig = make_subplots(specs=[[{"secondary_y": True}]])
            fig.add_trace(go.Scatter(x=aov_cpo['coupon'], y=aov_cpo['cpo_increment'], name="CPO Increment",line=dict(color="#ec1801"),yaxis="y"),secondary_y=True)
            fig.add_trace(go.Scatter(x=aov_cpo['coupon'], y=aov_cpo['order_increment'], name="Order Increment",line=dict(color="#0000ff"), yaxis="y2"),secondary_y=False)
            
            fig.update_xaxes(title_text="Coupon Group")
            fig.update_layout(title_text="Group AOV {} in {}".format(i,j),
                              yaxis=dict(title = 'CPO Increment',mirror= True, showline = True, ticklen=4,zeroline= False),
                              yaxis2=dict(title ='Orders Increment', overlaying='y', showgrid= False, side= 'right', ticklen=4,zeroline= False )
                             )
            
            fig.show()          

#### 